## Лабораторная работа №2
# "Методы поиска"
## Выполнила студентка группы БСТ1901 Леонова Полина
### Ход работы 
Реализовать методы поиска в соответствии с заданием. Организовать генерацию начального набора случайных данных. Для всех вариантов добавить реализацию добавления, поиска и удаления элементов. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

In [24]:
### Генерация списка
import random

def random_list(m=50, min=0, max=1000):
    return [random.randint(min, max) for _ in range(m)]

num = random_list(m=1000)

### Задание 1

#### Бинарный поиск
Основная идея - выбрать элемент из середины отсортированного массива (mid) и сравнить его с аргументом поиска.
Если он равен аргументу поиска, то поиск заканчивается. Если он меньше, чем аргумент поиска, то мы заключаем, что все элементы с индексами, меньшими, чем или равными mid можно исключить из дальнейшего поиска. Если элемент больше фргумента поиска, то поступаем аналогично.

In [25]:
def bin_search(arr, value):
    left = 0
    right = len(arr) - 1
    
    while left < right:
        mid = (left + right) // 2
        if arr[mid] < value:
            left = mid + 1
        else:
            right = mid
            
    return left

#### Бинарное дерево
Или двоичное дерево поиска - это бинарное дерево, обладающее дополнительными свойствами: значение первого потомка меньше значения родителя, а значение правого потоимка больше значения родителя для каждого узла дерева.(!БОЛЬШЕ-ВПРАВО, МЕНЬШЕ-ВЛЕВО!)


In [26]:
class Node:

    def __init__(self, data):

        self.left = None
        self.right = None
        self.data = data

    def insert(self, data):

        if self.data:
            if data < self.data:
                if self.left is None:
                    self.left = Node(data)
                else:
                    self.left.insert(data)
            elif data > self.data:
                if self.right is None:
                    self.right = Node(data)
                else:
                    self.right.insert(data) 
        else:
            self.data = data
            
    def findval(self, lkpval):
        if lkpval < self.data:
            if self.left is None:
                return str(lkpval)+" Not Found"
            return self.left.findval(lkpval)
        elif lkpval > self.data:
            if self.right is None:
                return str(lkpval)+" Not Found"
            return self.right.findval(lkpval)
        else:
            return self.data

    def PrintTree(self):
        if self.left:
            self.left.PrintTree()
        print( self.data),
        if self.right:
            self.right.PrintTree()

# Заполняем дерево
root = Node(num[0])
for m in range(1, len(num)):
    root.insert(num[m])

#### Интерполяционный поиск
Чем-то напоминает двоичный, только в нем область поиска не делится на 2 равные части, вместо этого производится оценка новой области поиска по расстоянию между текущими ззначениями между текущим элементом и ключом поиска. По скорости превосходит двоичный.
low -начальный индекс массива
high последний индекс массива 
m вероятный индекс искомого элемента


In [27]:
def interpolation(arr, val):
    low = 0 
    high = len(arr)-1 
    while low <= high and val >= arr[low] and val <= arr[high]:
        m = low + (high-low)*(val-arr[low])//(arr[high]-arr[low]) 
        if arr[m] == val: 
            return m 
        elif arr[m] < val:
            low = m+1 
        else: 
            high = m-1 
    return -1

#### Время работы

In [28]:
import time
functions = {
    'Бинарный поиск': bin_search,
    'Бинарное дерево': Node,
    'Интерполяционный поиск': interpolation,
}

for name, function in functions.items():
    time_start = time.perf_counter()
    if name == 'Бинарное дерево':
        root.findval(num[100])
    else:
        function(sorted(num), num[100])
    print(f'{name} - {time.perf_counter() - time_start}')

Бинарный поиск - 0.00048449999985678005
Бинарное дерево - 2.310000013494573e-05
Интерполяционный поиск - 0.0003897999999935564


### Задание №2
При построении хэш-таблиц сталкиваются с проблемой эффективного перевыделения памяти (такая же проблема и у динамических массивов) и проблемы коллизей хешей, поскольку все вычисления индексов идут в конечном поле с порядком равном размеру массива.
Проблема коллизий может решаться несколькими способами(для всех решений приходится хранить еще и ключ в ячейке для того , чтобы свериться) 
#### Простое рехеширование 
если произошла коллизия и ключи не совпали мы просто добавляем к индексу +1, пока не найдем свободную ячейку или ячейку с нужным ключом

In [29]:
class Hash:
    def __init__(self):
        self.data = []
        self.size = 0
        self._resize()

    def _hash(self, value, i):
        return (hash(value) + i) % len(self.data)

    def _resize(self):
        temp = self.data
        self.data = [None] * (2*len(self.data) + 1)
        for item in temp:
            if item is not None:
                self.data[self._rehash(item)] = item


    def _rehash(self, value):
        i = 0
        index = self._hash(value, i)

        while self.data[index] is not None:
            i += 1
            index = self._hash(value, i)

        return index

    def set_item(self, value):
        if self.size + 1 > len(self.data) // 2:
            self._resize()

        index = self._rehash(value)

        if self.data[index] is None:  
            self.size += 1

        self.data[index] = value

    def print_data(self):
        print(self.data)
        

####  Рехеширование с использованием случайных чисел
или псевдослучайных, в любом случае количество коллизий в будущем уменьшается , потому что даные хранятся более разрозненно

In [30]:
class RandomHash(Hash):
    _rand_num = [2309]

    def _rand(self, i):
        if len(self._rand_num) - 1 < i:
            self._rand_num.append(self._rand(i - 1))
        return (123456789 * self._rand_num[i] + 987654321) % 65546

    def _hash(self, value, i):
        return (hash(value) + self._rand(i)) % len(self.data)

#### Метод цепочек
в ячейках храним ссылки на динамические массивы, а уже в динамические масссивы мы либо добавляем элемент, либо меняем найденный

In [31]:
class ChainHash:
    def __init__(self):
        self.data = []
        self.size = 0
        self._resize()

    def _resize(self):
        temp = self.data
        self.data = [None] * (2*len(self.data) + 1)
        for item in temp:
            if item is not None:
                self.data[self._hash(item)] = item

    def _hash(self, value):
        print(hash(value) % len(self.data))
        return hash(value) % len(self.data)

    def set_item(self, value):
        if self.size + 1 > len(self.data) // 2:
            self._resize()

        index = self._hash(value)

        if self.data[index] is None:  
            self.size += 1
            self.data[index] = value
        else:
            if type(self.data[index]) == list:
                self.data[index].append(value)
            else:
                self.data[index] = [self.data[index]]
                self.data[index].append(value)

    def print_data(self):
        print(self.data)

### Задание №3
Расставить на стандартной 64-клеточной шахматной доске 8 ферзей так, чтобы ни один из них не находился под боем другого.
Подразумевается, что ферзь бьет все клетки, расположенные по вертикалям, горизонталям и обеим диагоналям.
Написать программу, которая находит хотя бы один способ решения задачи.

In [32]:
chess = [[0 for _ in range(8)] for _ in range(8)]

def can_stand(chess, row, column):
    i_p = 0
    i_m = 0
    for r in range(8):
        if chess[r][column] == 1:
            return False
    for c in range(column):
        if chess[row][c] == 1:
            return False
    for r in range(row, 8):
        if column + i_p >= 8: break
        if chess[r][column + i_p] == 1 or chess[r][column - i_p] == 1:
            return False
        i_p += 1
    for r in range(row, -1, -1):
        if column - i_m >= 8: break
        if chess[r][column + i_m] == 1 or chess[r][column - i_m] == 1:
            return False
        i_m -= 1
    return True


def queen(chess, row=0, column=0):
    if row > 7:
        return True
    for col in range(column, 8):
        if can_stand(chess, row, col):
            chess[row][col] = 1
            if col + 1 >= 7: col = 0
            queen(chess, row=row+1, column=col)
            break

queen(chess)
chess

[[1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1]]